<a href="https://colab.research.google.com/github/thesimaranjaira/AspectExtraction_SentimentAnalysis/blob/main/RandomForest_perhome_tcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
from termcolor import colored as cl
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/dataset/data_no1.csv')
#df = df.drop(['time', 'tempk', 'forecasttime'], axis=1)
print(df.head(10))
print(df.describe(include='all'))


             timestamp  consumption  ...  forecasttotalpulse  homesforecastedpulse
0  2018-08-13 01:00:00     1.167373  ...                 NaN                   NaN
1  2018-08-13 02:00:00     1.171019  ...                 NaN                   NaN
2  2018-08-13 03:00:00     1.102508  ...                 NaN                   NaN
3  2018-08-13 04:00:00     1.281203  ...                 NaN                   NaN
4  2018-08-13 05:00:00     1.562144  ...                 NaN                   NaN
5  2018-08-13 06:00:00     1.401059  ...                 NaN                   NaN
6  2018-08-13 07:00:00     1.229084  ...                 NaN                   NaN
7  2018-08-13 08:00:00     1.124097  ...                 NaN                   NaN
8  2018-08-13 09:00:00     1.095233  ...                 NaN                   NaN
9  2018-08-13 10:00:00     1.076520  ...                 NaN                   NaN

[10 rows x 22 columns]
                  timestamp  ...  homesforecastedpulse
count   

In [5]:
ActualLoad_total = (pd.read_csv("/content/drive/My Drive/dataset/data_no1.csv", index_col=["timestamp"], parse_dates=True).iloc[:, 2]).rename("load")
ForecastedLoad_home= (pd.read_csv("/content/drive/My Drive/dataset/data_no1.csv", index_col=["timestamp"], parse_dates=True).iloc[:, 10]).rename("forecasted_GLM")
tempC = (pd.read_csv("/content/drive/My Drive/dataset/data_no1.csv", index_col=["timestamp"], parse_dates=True).iloc[:, 4])
perHome= (pd.read_csv("/content/drive/My Drive/dataset/data_no1.csv", index_col=["timestamp"], parse_dates=True).iloc[:, 0]).rename("consumption_perHome")


In [6]:
print(ActualLoad_total, ForecastedLoad_home, tempC, perHome)

timestamp
2018-08-13 01:00:00      3190.430
2018-08-13 02:00:00      3200.395
2018-08-13 03:00:00      3013.154
2018-08-13 04:00:00      3501.527
2018-08-13 05:00:00      4269.340
                          ...    
2021-01-28 18:00:00    180122.345
2021-01-28 19:00:00    180508.608
2021-01-28 20:00:00    177324.554
2021-01-28 21:00:00    171417.187
2021-01-28 22:00:00    163884.793
Name: load, Length: 21598, dtype: float64 timestamp
2018-08-13 01:00:00    1.281439
2018-08-13 02:00:00    1.273370
2018-08-13 03:00:00    1.247555
2018-08-13 04:00:00    1.311142
2018-08-13 05:00:00    1.426472
                         ...   
2021-01-28 18:00:00    4.443659
2021-01-28 19:00:00    4.397501
2021-01-28 20:00:00    4.306531
2021-01-28 21:00:00    4.154426
2021-01-28 22:00:00    3.940354
Name: forecasted_GLM, Length: 21598, dtype: float64 timestamp
2018-08-13 01:00:00     7.182149
2018-08-13 02:00:00     6.393077
2018-08-13 03:00:00     6.117321
2018-08-13 04:00:00     6.660875
2018-08-13 05:00:0

In [7]:
def getSeason(perHome):
  season=pd.Series()
  
  for year in perHome.index.year.unique():

    yearlySeason = pd.Series("inter-season", index=perHome.loc[str(year)].index)

    yearlySeason.loc[: "{}-2-27".format(year)] ="winter"
    yearlySeason.loc["{}-11-1".format(year):] = "winter"
    yearlySeason.loc["{}-5-1".format(year): "{}-9-30".format(year)]= "summer"

    season=pd.concat([season, yearlySeason])
  return season

In [8]:
season= getSeason(perHome).rename("season")
print(season)

2018-08-13 01:00:00    summer
2018-08-13 02:00:00    summer
2018-08-13 03:00:00    summer
2018-08-13 04:00:00    summer
2018-08-13 05:00:00    summer
                        ...  
2021-01-28 18:00:00    winter
2021-01-28 19:00:00    winter
2021-01-28 20:00:00    winter
2021-01-28 21:00:00    winter
2021-01-28 22:00:00    winter
Name: season, Length: 21598, dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [9]:
def plot_daily(perHome, seasonStr):
  xaxis= pd.date_range("2018-8-13", periods=24, freq="1h").strftime("%H:%M")
  fontsize = 15
  df= perHome[season==seasonStr].to_frame()
  df= df.assign(day_name = df.index.day_name(), hour= df.index.strftime("%H:%M"))

  weekdayName= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

  (df.pivot_table("consumption_perHome", index="day_name", columns= "hour")
    .T
    .loc[: , weekdayName]
    .plot(figsize=[8,5],
          lw=3));

  plt.xticks(np.arange(0, len(xaxis), 2), xaxis[::2]);
  plt.xlabel("Hour", fontsize=fontsize)
  plt.ylabel("Load", fontsize=fontsize);
  plt.grid()
  plt.title("Mean Daily Profile- {}".format(seasonStr), fontsize=fontsize+5);
  plt.ylim(10,23)
  plt.autoscale()

In [10]:
def plot_season(x,y, xlabel, ylabel, title, freq, s=30):
  df= pd.concat([x,y],axis=1)

  seasons=["winter", "inter-season", "summer"]
  colors= {"winter":blue, "inter-season":grey, "summer":red}

  fig= plt.figure(figsize=[17,5])
  size=12
  alpha=0.5

  for i,seasonStr in enumerate(seasons):

    fig.add_subplot(1, 3, i+1)
    df[season==seasonStr].resample(freq).mean().plot.scatter(ax=plt.gca(), 
                                                              x= x.name, y=y.name, s=s, alpha=alpha, color=colors[seasonStr]);

    plt.grid()
    plt.xlabel(xlabel, size=size)
    plt.ylabel(ylabel, size=size)
    plt.title("{} - {}".format(seasonStr.capitalize(),title), size=size);
    plt.xticks(size=size, rotation=45), plt.yticks(size=size);
    plt.autoscale()
  plt.tight_layout()

In [11]:
def get_interactions(s, dummies, poly_degree=1):
    if s.name == None: s.name = "_"
    s1 = dummies.apply(lambda x: x * s).rename("{}".format(s.name + "_{}").format, axis=1)
    s2 = (dummies.apply(lambda x: x * s) **2).rename("{}".format(s.name + "2_{}").format, axis=1)
    s3 = (dummies.apply(lambda x: x * s) **3).rename("{}".format(s.name + "3_{}").format, axis=1)
    
    if poly_degree == 1: s2, s3 = None, None
    if poly_degree == 2: s3 = None

    df = pd.concat([s1, s2, s3], axis=1)
    return df


In [12]:
def get_season(ts_in):
    season = pd.Series()

    for year in ts_in.index.year.unique():

        # CREATE SEASONS
        yearly_season = pd.Series("inter-season", index=ts_in.loc[str(year)].index)

        yearly_season.loc[: "{}-2-15".format(year)] = "winter"
        yearly_season.loc["{}-11-15".format(year) :] = "winter"
        yearly_season.loc["{}-5-15".format(year) : "{}-9-15".format(year)] = "summer"

        season = pd.concat([season, yearly_season])
    return season


In [13]:
y_home= perHome.rename("y_home").copy()

In [14]:
season = get_season(y_home).rename("season")
month = pd.Series(y_home.index.month.astype(str), index=y_home.index, name="month").apply(lambda x: "m{}".format(x))
day = pd.Series(y_home.index.dayofweek.astype(str), index=y_home.index, name="day").apply(lambda x: "d{}".format(x))
hour = pd.Series(y_home.index.strftime("%H:%M"), index=y_home.index, name="hour")
dayhour = (day + "_" + hour).rename("dayhour")

month_dummies = pd.get_dummies(month.sort_values()).sort_index()
day_dummies = pd.get_dummies(day.sort_values()).sort_index()
hour_dummies = pd.get_dummies(hour.sort_values()).sort_index()
dayhour_dummies = pd.get_dummies(dayhour.sort_values()).sort_index()

trend = pd.Series(np.arange(1, len(y_home) + 1), y_home.index, name="trend")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [15]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

**Features: Hour of the Day**

Without dropping intercept and dummy variables.

In [78]:
X = hour_dummies.values
y = y_home
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    # for mf in np.linspace(100, 150, 6):
    #     for ne in np.linspace(50, 100, 6):
    #         for md in np.linspace(20, 40, 5):
    #             for msl in np.linspace(30, 100, 8):
    #                 rfr = RandomForestRegressor(
    #                     max_features= int(mf),
    #                     n_estimators=int(ne),
    #                     max_depth=int(md),
    #                     min_samples_leaf=int(msl))
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    #                 score.append([i, mf, ne, md, msl,rfr.score(X_val, y_val)])
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_hour=yhat.copy()

38.36876202810531
[[1, -4.121354348377801], [2, -5.4510033283489765], [3, -0.03152669137266462], [4, -7.719248352291123], [5, -16.53491638244411], [6, 0.14784535041634161], [7, -3.4710240936771206], [8, -1.4565095809794193], [9, -0.23066468883052882], [10, -20.857080318981097]]


In [79]:
#timeseries cross validation
X = hour_dummies.values
y = y_                                1q2home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_hour=yhat.copy()

38.36978774615606
[[1, -4.110702643768136], [2, -5.455574700523615], [3, -0.031221694457535643], [4, -7.7124785923837], [5, -16.549737611711343], [6, 0.14870474522408317], [7, -3.4846683993059044], [8, -1.45050536319068], [9, -0.2332487695316856], [10, -20.879935518315264]]


In [75]:
#timeseries cross validation
X = hour_dummies.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = LinearRegression()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_hour=yhat.copy()


38.365871996325055
[[1, -4.1118143657542605], [2, -5.453728207489228], [3, -0.03224134384301225], [4, -7.720702897477768], [5, -16.505817891302215], [6, 0.14817656824866365], [7, -3.4734146642788004], [8, -1.4554422772135465], [9, -0.23660754453573293], [10, -20.86245766459601]]


**Features: Month + Day of the Week + Hour of the Day**

Dropping one dummy to avoid colinearity

In [68]:
#time series cross val
Xhat = pd.concat([month_dummies.iloc[:, :-1],  
               day_dummies.iloc[:, :-1],   
               hour_dummies.iloc[:, :-1]    
              ], axis=1)
X = Xhat.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_mdh = yhat.copy() 

13.908409504873035
[[1, -3.946665740056206], [2, -12.173281357880631], [3, -1.6065438684144477], [4, 0.20165945745594027], [5, 0.5460393460471826], [6, 0.8057567658887066], [7, -0.4136040244508119], [8, 0.07746731727964984], [9, 0.32607666652364886], [10, 0.16150601995149758]]


In [60]:
#time series cross val
Xhat = pd.concat([month_dummies.iloc[:, :-1],  
               day_dummies.iloc[:, :-1],   
               hour_dummies.iloc[:, :-1]    
              ], axis=1)
X = Xhat.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = LinearRegression()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_mdh = yhat.copy() 

14.560776873544953
[[1, -1.412567864546294e+23], [2, -6.199036167622866e+22], [3, -4.305074184473245e+24], [4, -2.2678055173820315e+23], [5, 0.4242096421267123], [6, 0.7672639182410638], [7, -0.4984185194371258], [8, -0.011444333321812605], [9, 0.3885473903503118], [10, 0.04856854908256636]]


**Features: Month + Day of the Week x Hour of the Day**

In [70]:
#with times sries cross val
X_hat = pd.concat([month_dummies.iloc[:, :-1], 
               dayhour_dummies.iloc[:, :-1],    
              ], axis=1)
X = X_hat.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat= pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_mdxh = yhat.copy() 

13.905440815081171
[[1, -3.9672807724779893], [2, -12.232436807950766], [3, -1.6206717120041312], [4, 0.2580334636969879], [5, 0.6670603396515884], [6, 0.8011630266321973], [7, -0.37468093171863437], [8, 0.08976231892940711], [9, 0.321230847806373], [10, 0.15425794072310706]]


In [57]:
#with times sries cross val
X_hat = pd.concat([month_dummies.iloc[:, :-1], 
               dayhour_dummies.iloc[:, :-1],    
              ], axis=1)
X = X_hat.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = LinearRegression()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat= pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_mdxh = yhat.copy() 

14.097548207413354
[[1, -5.430659737096881e+23], [2, -5.004521356274457e+25], [3, -3.7681811620337145e+26], [4, -6.169639764789976e+21], [5, 0.4623163888857131], [6, 0.8008820608282936], [7, -0.4324918589831286], [8, 0.09340578577547498], [9, 0.3980268741141967], [10, 0.04889424040619583]]


**Modelling Temperature**

Features used is a 3rd Degree Polynomial because it can model assymteric depdendencies.


($temp$ x $month$ + $temp^{2}$ x $month$ + $temp^{3}$ x $month$) +
($temp$ x $hour$ + $temp^{2}$ x $hour$ + $temp^{3}$ x $hour$) +
($month$ + $day\_of\_the\_week$ x $hour\_of\_the\_day$)

In [69]:
#with time series cross val
X_HAT = pd.concat([month_dummies.iloc[:, :-1],  
               dayhour_dummies.iloc[:, :-1],    
               # month interactions
               get_interactions(tempC, month_dummies, poly_degree=3),
               # hourly interactions
               get_interactions(tempC, hour_dummies, poly_degree=3)   
              ], axis=1)
X = X_HAT.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_temp = yhat.copy() 

7.712862651732033
[[1, -5.547475120168803], [2, -16.72445714131413], [3, -1.0660023885042933], [4, -6.465166513733013], [5, 0.7129628912824457], [6, 0.8812549350538604], [7, 0.4136146699864236], [8, 0.5032652204329141], [9, 0.6385970744890617], [10, 0.6813752918507079]]


In [54]:
#with time series cross val
X_HAT = pd.concat([month_dummies.iloc[:, :-1],  
               dayhour_dummies.iloc[:, :-1],    
               # month interactions
               get_interactions(tempC, month_dummies, poly_degree=3),
               # hourly interactions
               get_interactions(tempC, hour_dummies, poly_degree=3)   
              ], axis=1)
X = X_HAT.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = LinearRegression()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_temp = yhat.copy() 

7.0619912308277915
[[1, -9.146012237583985e+20], [2, -8.05112562591895e+19], [3, -2.366255406101543e+20], [4, -3.4038009467744335e+23], [5, 0.5929276763872058], [6, 0.9317935344614438], [7, 0.5964595117689045], [8, 0.6369825636021345], [9, 0.662830583348198], [10, 0.596097634427267]]


**Modelling Trends**

Adding a trend feature

In [71]:
#with time series cross val
X_h = pd.concat([month_dummies.iloc[:, :-1],  
               dayhour_dummies.iloc[:, :-1],   
               
               get_interactions(tempC, month_dummies, poly_degree=3),  # month interactions
               get_interactions(tempC, hour_dummies, poly_degree=3),   # hourly interactions
               
               trend
              ], axis=1)
X = X_h.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_trend = yhat.copy() 


14.599127725675565
[[1, -2.480172631207723], [2, 0.3478083452981644], [3, -0.3109684402973436], [4, -0.06648677207020426], [5, 0.24404596970864065], [6, -0.9297262510788178], [7, -0.05615169741111536], [8, 0.38935051223182804], [9, 0.04398649864685644], [10, 0.4682318799693565]]


In [81]:
#with time series cross val
X_h = pd.concat([month_dummies.iloc[:, :-1],  
               dayhour_dummies.iloc[:, :-1],   
               
               get_interactions(tempC, month_dummies, poly_degree=3),  # month interactions
               get_interactions(tempC, hour_dummies, poly_degree=3),   # hourly interactions
               
               trend
              ], axis=1)
X = X_h.values
y = y_home
#y = y_home.values.reshape(-1, 1)
X_train = X[:int(X.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]
tscv = TimeSeriesSplit(n_splits=10)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = LinearRegression()
    rfr.fit(X_tr, y_tr)
    score.append([i, rfr.score(X_val, y_val)])
    i += 1
yhat=pd.Series(rfr.predict(X_test))
MAPE= mean_absolute_percentage_error(y_test,yhat)
print(MAPE)
print(score)
y_pred_trend = yhat.copy()



6.748232502426074
[[1, -4.990967869584451e+20], [2, -8.206862970493237e+21], [3, -5.217702371383096e+20], [4, -3.938699921396484], [5, 0.603453541349234], [6, 0.9226348041094488], [7, 0.6364539784873081], [8, 0.7121369514065499], [9, 0.5569769120100403], [10, 0.4641706934309959]]
